### Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis = 1)
appl_count = application_df["APPLICATION_TYPE"].value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(appl_count[appl_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Look at CLASSIFICATION value counts for binning
classification_count = application_df["CLASSIFICATION"].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Generate our categorical variable lists
application_cat = list(application_df.dtypes[application_df.dtypes == "object"].index)

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index = True, right_index = True)
application_df = application_df.drop(columns = application_cat)

# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns = "IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


C:\Users\UNKNOWN\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Deliverable 2: Compile, Train and Evaluate the Model
#### Increasing Epochs to 150

In [2]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, activation = "sigmoid", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1,activation = "linear"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [3]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [6]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs = 150)

Epoch 1/150
804/804 [==============================] - 3s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 2/150
804/804 [==============================] - 2s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 3/150
804/804 [==============================] - 2s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 4/150
804/804 [==============================] - 2s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 5/150
804/804 [==============================] - 2s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 7/150
804/804 [==============================] - 3s 4ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 8/150
804/804 [==============================] - 3s 4ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 9/150
804/804 [==============================] - 3s 4ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 10/150
804/804 [==============================] - 3s 3ms/step - loss: 7.1403 - accura

804/804 [==============================] - 2s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 82/150
804/804 [==============================] - 2s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 83/150
804/804 [==============================] - 3s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 84/150
804/804 [==============================] - 2s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 85/150
804/804 [==============================] - 2s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 86/150
804/804 [==============================] - 3s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 87/150
804/804 [==============================] - 2s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 88/150
804/804 [==============================] - 3s 3ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 89/150
804/804 [==============================] - 3s 4ms/step - loss: 7.1403 - accuracy: 0.5318
Epoch 90/150
804/804 [==============================] - 2s 3ms/step - loss: 7.1403 - accuracy: 

In [7]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 2s - loss: 2.3589 - accuracy: 0.5662 - 2s/epoch - 9ms/step
Loss: 2.358891725540161, Accuracy: 0.5661807656288147


#### Changing Activation functions

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1,activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [10]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 31803.0547 - accuracy: 0.5043
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 24967.3105 - accuracy: 0.4844
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 5754.2720 - accuracy: 0.4886
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 11700.9912 - accuracy: 0.5087
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 10415.5996 - accuracy: 0.4843
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 7289.4399 - accuracy: 0.4948
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 3371.0593 - accuracy: 0.5230
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 3040.0442 - accuracy: 0.5109
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 1154.1201 - accuracy: 0.5168
Epoch 10/100
804/804 [==============================] - 3s 3

804/804 [==============================] - 2s 2ms/step - loss: 0.6915 - accuracy: 0.5318
Epoch 81/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5318
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 85/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6915 - accuracy: 0.5318
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6914 - accuracy: 0.5318
Epoch 89/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6913 - accuracy: 

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 1.3447 - accuracy: 0.6660 - 818ms/epoch - 3ms/step
Loss: 1.3447188138961792, Accuracy: 0.6660058498382568


#### Dropping more columns

In [12]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")

# Drop the non-beneficial ID columns, 'EIN' and 'NAME','STATUS','SPECIAL_CONSIDERATIONS'.
application_df = application_df.drop(["EIN", "NAME","STATUS","SPECIAL_CONSIDERATIONS"], axis = 1)
appl_count = application_df["APPLICATION_TYPE"].value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(appl_count[appl_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Look at CLASSIFICATION value counts for binning
classification_count = application_df["CLASSIFICATION"].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Generate our categorical variable lists
application_cat = list(application_df.dtypes[application_df.dtypes == "object"].index)

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index = True, right_index = True)
application_df = application_df.drop(columns = application_cat)

# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns = "IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


C:\Users\UNKNOWN\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, activation = "sigmoid", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1,activation = "linear"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3280      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,741
Trainable params: 5,741
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs = 150)

Epoch 1/150
804/804 [==============================] - 3s 3ms/step - loss: 0.7023 - accuracy: 0.5111
Epoch 2/150
804/804 [==============================] - 2s 3ms/step - loss: 0.7002 - accuracy: 0.5086
Epoch 3/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6995 - accuracy: 0.5154
Epoch 4/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6995 - accuracy: 0.5133
Epoch 5/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6965 - accuracy: 0.5161
Epoch 6/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6989 - accuracy: 0.5150
Epoch 7/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6963 - accuracy: 0.5091
Epoch 8/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6967 - accuracy: 0.5181
Epoch 9/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6960 - accuracy: 0.5174
Epoch 10/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6946 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.6918 - accuracy: 0.5318
Epoch 82/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6918 - accuracy: 0.5304
Epoch 83/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6918 - accuracy: 0.5288
Epoch 84/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6916 - accuracy: 0.5274
Epoch 85/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6919 - accuracy: 0.5290
Epoch 86/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6917 - accuracy: 0.5309
Epoch 87/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6916 - accuracy: 0.5290
Epoch 88/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6919 - accuracy: 0.5310
Epoch 89/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6917 - accuracy: 0.5291
Epoch 90/150
804/804 [==============================] - 2s 3ms/step - loss: 0.6916 - accuracy: 

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Saving to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")